In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [3]:
df=pd.read_csv("vocal_gender_features_new.csv")

In [4]:
df["label"].value_counts()


label
1    10380
0     5768
Name: count, dtype: int64

In [5]:
df.drop_duplicates(inplace=True)


In [ ]:
df

In [6]:
df.shape

(15070, 44)

In [7]:
df["label"].value_counts()


label
1    9841
0    5229
Name: count, dtype: int64

In [8]:
skew_vals = df.skew()
high_skew_cols = skew_vals[skew_vals.abs() > 0.75]
print(high_skew_cols)


mean_spectral_contrast    -0.899611
min_pitch                  4.878715
max_pitch                -13.400446
std_pitch                 -1.434120
spectral_kurtosis          1.478030
mfcc_8_std                 0.989087
dtype: float64


In [9]:
df['mfcc_8_std'] = np.sqrt(df['mfcc_8_std'])
df['mean_spectral_contrast'] = df['mean_spectral_contrast'] ** 2
df['min_pitch'] = np.log1p(df['min_pitch'] - df['min_pitch'].min() + 1)
df['spectral_kurtosis'] = np.log1p(df['spectral_kurtosis'] - df['spectral_kurtosis'].min() + 1)
df['max_pitch'] = df['max_pitch'] ** 3
df['std_pitch'] = df['std_pitch'] ** 3


In [10]:
X = df.drop("label", axis=1)
y = df["label"]


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


random forest

In [13]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    class_weight="balanced",   # 🔥 handles imbalance
    random_state=42
)

rf.fit(X_train_scaled, y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",300
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [14]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = rf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.9943596549435966

Classification Report:

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1046
           1       0.99      1.00      1.00      1968

    accuracy                           0.99      3014
   macro avg       1.00      0.99      0.99      3014
weighted avg       0.99      0.99      0.99      3014


Confusion Matrix:

[[1032   14]
 [   3 1965]]


In [15]:
importances = pd.Series(rf.feature_importances_, index=X.columns)

# Sort descending
importances = importances.sort_values(ascending=False)

# Get top 20
top_20_features = importances.head(20)

print(top_20_features)

mfcc_5_mean               0.157856
mean_spectral_contrast    0.098536
mfcc_3_std                0.087291
mfcc_2_mean               0.073235
std_spectral_bandwidth    0.050529
mfcc_12_mean              0.046632
mfcc_1_mean               0.039390
mfcc_2_std                0.032461
mfcc_10_mean              0.031611
rms_energy                0.030986
mfcc_10_std               0.025872
mfcc_6_mean               0.024496
mfcc_8_mean               0.023265
mfcc_4_mean               0.022135
mfcc_13_mean              0.020578
mfcc_7_mean               0.020561
mfcc_3_mean               0.016078
mfcc_8_std                0.014437
mfcc_11_mean              0.014035
mfcc_5_std                0.013973
dtype: float64


MFCC features capture:

Timbre

Vocal tract characteristics

Resonance patterns

Gender differences are not just pitch — they are also:

Vocal tract length

Harmonic structure

Frequency envelope shape

So model is learning deeper acoustic structure

svm

In [16]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [17]:
svm = SVC(
    kernel="rbf", 
    class_weight="balanced",
    probability=True,
    random_state=42
)

svm.fit(X_train_scaled, y_train)


,"C C: float, default=1.0Regularization parameter. The strength of the regularization isinversely proportional to C. Must be strictly positive. The penaltyis a squared l2 penalty. For an intuitive visualization of the effectsof scaling the regularization parameter C, see:ref:`sphx_glr_auto_examples_svm_plot_svm_scale_c.py`.",1.0
,"kernel kernel: {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'} or callable, default='rbf'Specifies the kernel type to be used in the algorithm. Ifnone is given, 'rbf' will be used. If a callable is given it is used topre-compute the kernel matrix from data matrices; that matrix should bean array of shape ``(n_samples, n_samples)``. For an intuitivevisualization of different kernel types see:ref:`sphx_glr_auto_examples_svm_plot_svm_kernels.py`.",'rbf'
,"degree degree: int, default=3Degree of the polynomial kernel function ('poly').Must be non-negative. Ignored by all other kernels.",3
,"gamma gamma: {'scale', 'auto'} or float, default='scale'Kernel coefficient for 'rbf', 'poly' and 'sigmoid'.- if ``gamma='scale'`` (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,- if 'auto', uses 1 / n_features- if float, must be non-negative... versionchanged:: 0.22 The default value of ``gamma`` changed from 'auto' to 'scale'.",'scale'
,"coef0 coef0: float, default=0.0Independent term in kernel function.It is only significant in 'poly' and 'sigmoid'.",0.0
,"shrinking shrinking: bool, default=TrueWhether to use the shrinking heuristic.See the :ref:`User Guide `.",True
,"probability probability: bool, default=FalseWhether to enable probability estimates. This must be enabled priorto calling `fit`, will slow down that method as it internally uses5-fold cross-validation, and `predict_proba` may be inconsistent with`predict`. Read more in the :ref:`User Guide `.",True
,"tol tol: float, default=1e-3Tolerance for stopping criterion.",0.001
,"cache_size cache_size: float, default=200Specify the size of the kernel cache (in MB).",200
,"class_weight class_weight: dict or 'balanced', default=NoneSet the parameter C of class i to class_weight[i]*C forSVC. If not given, all classes are supposed to haveweight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.",'balanced'
,"verbose verbose: bool, default=FalseEnable verbose output. Note that this setting takes advantage of aper-process runtime setting in libsvm that, if enabled, may not workproperly in a multithreaded context.",False


In [18]:
y_pred_svm = svm.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_svm))
print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred_svm))


Accuracy: 0.9993364299933643

Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1046
           1       1.00      1.00      1.00      1968

    accuracy                           1.00      3014
   macro avg       1.00      1.00      1.00      3014
weighted avg       1.00      1.00      1.00      3014


Confusion Matrix:

[[1045    1]
 [   1 1967]]


svm cross validation

In [19]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(svm, X_train_scaled, y_train, cv=5)

print("Cross-validation accuracy:", scores)
print("Mean CV accuracy:", scores.mean())


Cross-validation accuracy: [0.99875622 0.99958523 0.99958523 0.99958523 0.99792617]
Mean CV accuracy: 0.9990876187292488


logistic regression

In [20]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(
    class_weight="balanced",
    max_iter=1000
)

lr.fit(X_train_scaled, y_train)

y_pred_lr = lr.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_lr))
print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred_lr))


Accuracy: 0.9913735899137359

Classification Report:

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1046
           1       1.00      0.99      0.99      1968

    accuracy                           0.99      3014
   macro avg       0.99      0.99      0.99      3014
weighted avg       0.99      0.99      0.99      3014


Confusion Matrix:

[[1039    7]
 [  19 1949]]


mlp classifier

In [21]:
from sklearn.neural_network import MLPClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [22]:
sample_weights = compute_sample_weight(
    class_weight="balanced",
    y=y_train
)


In [23]:
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42
)


In [24]:
mlp.fit(X_train_scaled, y_train, sample_weight=sample_weights)


,"hidden_layer_sizes hidden_layer_sizes: array-like of shape(n_layers - 2,), default=(100,)The ith element represents the number of neurons in the ithhidden layer.","(64, ...)"
,"activation activation: {'identity', 'logistic', 'tanh', 'relu'}, default='relu'Activation function for the hidden layer.- 'identity', no-op activation, useful to implement linear bottleneck, returns f(x) = x- 'logistic', the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).- 'tanh', the hyperbolic tan function, returns f(x) = tanh(x).- 'relu', the rectified linear unit function, returns f(x) = max(0, x)",'relu'
,"solver solver: {'lbfgs', 'sgd', 'adam'}, default='adam'The solver for weight optimization.- 'lbfgs' is an optimizer in the family of quasi-Newton methods.- 'sgd' refers to stochastic gradient descent.- 'adam' refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy BaFor a comparison between Adam optimizer and SGD, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_training_curves.py`.Note: The default solver 'adam' works pretty well on relativelylarge datasets (with thousands of training samples or more) in terms ofboth training time and validation score.For small datasets, however, 'lbfgs' can converge faster and performbetter.",'adam'
,"alpha alpha: float, default=0.0001Strength of the L2 regularization term. The L2 regularization termis divided by the sample size when added to the loss.For an example usage and visualization of varying regularization, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_alpha.py`.",0.0001
,"batch_size batch_size: int, default='auto'Size of minibatches for stochastic optimizers.If the solver is 'lbfgs', the classifier will not use minibatch.When set to ""auto"", `batch_size=min(200, n_samples)`.",'auto'
,"learning_rate learning_rate: {'constant', 'invscaling', 'adaptive'}, default='constant'Learning rate schedule for weight updates.- 'constant' is a constant learning rate given by 'learning_rate_init'.- 'invscaling' gradually decreases the learning rate at each time step 't' using an inverse scaling exponent of 'power_t'. effective_learning_rate = learning_rate_init / pow(t, power_t)- 'adaptive' keeps the learning rate constant to 'learning_rate_init' as long as training loss keeps decreasing. Each time two consecutive epochs fail to decrease training loss by at least tol, or fail to increase validation score by at least tol if 'early_stopping' is on, the current learning rate is divided by 5.Only used when ``solver='sgd'``.",'constant'
,"learning_rate_init learning_rate_init: float, default=0.001The initial learning rate used. It controls the step-sizein updating the weights. Only used when solver='sgd' or 'adam'.",0.001
,"power_t power_t: float, default=0.5The exponent for inverse scaling learning rate.It is used in updating effective learning rate when the learning_rateis set to 'invscaling'. Only used when solver='sgd'.",0.5
,"max_iter max_iter: int, default=200Maximum number of iterations. The solver iterates until convergence(determined by 'tol') or this number of iterations. For stochasticsolvers ('sgd', 'adam'), note that this determines the number of epochs(how many times each data point will be used), not the number ofgradient steps.",500
,"shuffle shuffle: bool, default=TrueWhether to shuffle samples in each iteration. Only used whensolver='sgd' or 'adam'.",True
,"random_state random_state: int, RandomState instance, default=NoneDetermines random number generation for weights and biasinitialization, train-test split if early stopping is used, and batchsampling when solver='sgd' or 'adam'.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",42


In [25]:
y_pred_mlp = mlp.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_mlp))
print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred_mlp))


Accuracy: 0.9986728599867286

Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1046
           1       1.00      1.00      1.00      1968

    accuracy                           1.00      3014
   macro avg       1.00      1.00      1.00      3014
weighted avg       1.00      1.00      1.00      3014


Confusion Matrix:

[[1045    1]
 [   3 1965]]


xgboost

In [26]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Calculate imbalance ratio
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

xgb.fit(X_train_scaled, y_train)

y_pred_xgb = xgb.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))


c:\Users\ADMIN\Desktop\human voice recognition\human\Lib\site-packages\xgboost\training.py:200: UserWarning: [15:15:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.9953550099535501
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1046
           1       1.00      1.00      1.00      1968

    accuracy                           1.00      3014
   macro avg       0.99      1.00      0.99      3014
weighted avg       1.00      1.00      1.00      3014

[[1040    6]
 [   8 1960]]


light gbm

In [27]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(
    n_estimators=300,
    class_weight='balanced',
    random_state=42
)

lgbm.fit(X_train_scaled, y_train)



[LightGBM] [Info] Number of positive: 7873, number of negative: 4183
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004719 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10965
[LightGBM] [Info] Number of data points in the train set: 12056, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,300
,subsample_for_bin,200000
,objective,None
,class_weight,'balanced'
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [28]:
y_pred_lgbm = lgbm.predict(X_test_scaled)
y_prob_lgbm = lgbm.predict_proba(X_test_scaled)[:,1]


c:\Users\ADMIN\Desktop\human voice recognition\human\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\ADMIN\Desktop\human voice recognition\human\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [29]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_lgbm))
print("Accuracy:", accuracy_score(y_test, y_pred_lgbm))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1046
           1       1.00      1.00      1.00      1968

    accuracy                           1.00      3014
   macro avg       1.00      1.00      1.00      3014
weighted avg       1.00      1.00      1.00      3014

Accuracy: 0.9970139349701393


WITH TOP 20 FEATURES

In [30]:
selected_features = [
    "mfcc_5_mean",
    "mean_spectral_contrast",
    "mfcc_3_std",
    "mfcc_2_mean",
    "std_spectral_bandwidth",
    "mfcc_12_mean",
    "mfcc_1_mean",
    "mfcc_2_std",
    "mfcc_10_mean",
    "rms_energy",
    "mfcc_10_std",
    "mfcc_6_mean",
    "mfcc_8_mean",
    "mfcc_4_mean",
    "mfcc_13_mean",
    "mfcc_7_mean",
    "mfcc_3_mean",
    "mfcc_8_std",
    "mfcc_11_mean",
    "mfcc_5_std"
]

X_top20 = X[selected_features]


In [31]:
from sklearn.model_selection import train_test_split

X_train_20, X_test_20, y_train, y_test = train_test_split(
    X_top20,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [32]:
from sklearn.preprocessing import StandardScaler

scaler_20 = StandardScaler()

X_train_20_scaled = scaler_20.fit_transform(X_train_20)
X_test_20_scaled = scaler_20.transform(X_test_20)


svm top 20

In [33]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

svm_20 = SVC(kernel="rbf", class_weight="balanced")

svm_20.fit(X_train_20_scaled, y_train)

y_pred_20 = svm_20.predict(X_test_20_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred_20))
print(classification_report(y_test, y_pred_20))
print(confusion_matrix(y_test, y_pred_20))


Accuracy: 0.9990046449900465
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1046
           1       1.00      1.00      1.00      1968

    accuracy                           1.00      3014
   macro avg       1.00      1.00      1.00      3014
weighted avg       1.00      1.00      1.00      3014

[[1045    1]
 [   2 1966]]


Reducing from 40+ features → 20 features:

 Did NOT hurt performance
 Maintained balanced recall
 Maintained near-perfect accuracy
 Reduced model complexity
 Reduced UI input size

In [34]:
import os

os.makedirs("models")


In [35]:
import joblib

joblib.dump(svm_20, "models/svm_20.pkl")
joblib.dump(scaler_20, "models/scaler_20.pkl")
joblib.dump(selected_features, "models/selected_features.pkl")



['models/selected_features.pkl']

In [36]:
import joblib


joblib.dump(selected_features, "models/top_20_features.pkl")

['models/top_20_features.pkl']

In [39]:
X_top20 = X[selected_features]


In [40]:
X_top20.to_csv("X_top20.csv", index=False)

In [41]:
y.to_csv("y.csv",index=False)